In [ ]:
!pip install dash==1.10.0
!pip install dash dash-renderer dash-html-components dash-core-components plotly
!pip install plotly
!pip install dash-bootstrap-components

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math
import os
import base64
import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

In [ ]:

###################################
# Private function
###################################


def load_data(file_name, column_name):
    data = (
        pd.read_csv(file_name,)
        .melt(
            id_vars=["Province/State", "Country/Region", "Lat", "Long"],
            var_name="date",
            value_name=column_name,
        )
        .astype({"date": "str", column_name: int}, errors="ignore")
    )
    data["Province/State"].fillna("<all>", inplace=True)
    data[column_name].fillna(0, inplace=True)
    return data
# def loadData(fileName, columnName):
#     data = pd.read_csv(fileName) \
#              .drop(['Lat', 'Long'], axis=1) \
#              .melt(id_vars=['Province/State', 'Country/Region'], 
#                  var_name='date', value_name=columnName) \
#              .astype({'date':'datetime64[ns]', columnName:'Int64'}, 
#                  errors='ignore')
#     data['Province/State'].fillna('<all>', inplace=True)
#     data[columnName].fillna(0, inplace=True)
#     return data



def create_card(title, content, change, currently, color):
    card = dbc.Card(
        dbc.CardBody(
            [
                html.H6(title, className="card-subtitle"),
                html.H4(content, className="card-subtitle"),
                html.P(change, className="card-subtitle"),
                html.P(currently, className="card-subtitle"),
            ],
            className="text-center",
        ),
        style={"color": color},
    )
    return card


def sum_data(data_frame, date, days_offset, column):
    sum = data_frame.loc[
        data_frame["date"] == date + pd.DateOffset(days_offset), column
    ].sum()
    return sum

In [ ]:
################################################################################
# Data processing
################################################################################

# Data and files working with
confirmed_cases = "time_series_covid19_confirmed_global.csv"
confirmed_deaths = "time_series_covid19_deaths_global.csv"
confirmed_recovered = "time_series_covid19_recovered_global.csv"

# Merge data frames into one
all_data = (
    load_data(confirmed_cases, "confirmed")
    .merge(load_data(confirmed_deaths, "deaths"))
    .merge(load_data(confirmed_recovered, "recovered"))
)

countries = all_data["Country/Region"].unique()
countries.sort()
# print(all_data)